# Phase 2 Demo: Complete MWB Tracking Pipeline

This notebook demonstrates the complete Phase 2 pipeline:
1. **Ingestion**: Load raw conversational text
2. **Preprocessing**: Normalize slang, emojis, and apply fixes
3. **Contextualization**: Create multi-turn sequences with context
4. **Modeling**: Emotion classification and intensity prediction
5. **Persistence**: Store results in SQLite

## Setup


In [1]:
import sys
from pathlib import Path

# Add src to path
project_root = Path.cwd().parent
sys.path.insert(0, str(project_root))

import pandas as pd
import logging
from datetime import datetime
import time

from src.pipeline import run_full_pipeline
from src.modeling import EmotionModel
from src.contextualize import create_sequences_batch
from src.persistence import MWBPersistence
from src.validation import AmbiguityGenerator, MetricsTracker
from src.utils import setup_logging

# Setup logging
setup_logging(log_level="INFO")
logging.info("Starting Phase 2 Demo")


2025-12-06 12:57:38,727 - root - INFO - Starting Phase 2 Demo


## Step 1-5: Complete Pipeline

Run the full pipeline from ingestion to persistence.


In [2]:
# Run complete pipeline
data_path = project_root / "data" / "sample_data.csv"
db_path = project_root / "data" / "mwb_log.db"
slang_dict_path = project_root / "data" / "slang_dictionary.json"
checkpoint_dir = project_root / "checkpoints"

print("Running complete Phase 2 pipeline...")
print("This may take a few minutes to download the model on first run.\n")

start_time = time.time()
df_results = run_full_pipeline(
    input_path=data_path,
    db_path=db_path,
    slang_dict_path=slang_dict_path,
    model_name="j-hartmann/emotion-english-distilroberta-base",
    max_context_turns=5,
    batch_size=16,
    checkpoint_dir=checkpoint_dir,
    archive_raw=True
)
elapsed = time.time() - start_time

print(f"\n✓ Pipeline complete in {elapsed:.2f} seconds")
print(f"✓ Processed {len(df_results)} records")


2025-12-06 12:57:38,732 - src.pipeline - INFO - Starting Phase 2 full pipeline
2025-12-06 12:57:38,732 - src.pipeline - INFO - [1/5] Ingestion...
2025-12-06 12:57:38,732 - src.ingest - INFO - Ingesting CSV from /Users/vikranthnara/Documents/GitHub/Emotix/data/sample_data.csv
2025-12-06 12:57:38,736 - src.ingest - INFO - Validated DataFrame with 20 rows
2025-12-06 12:57:38,770 - src.utils - INFO - Checkpoint saved: /Users/vikranthnara/Documents/GitHub/Emotix/checkpoints/ingestion_20251206_125738.parquet (20 rows)
2025-12-06 12:57:38,770 - src.pipeline - INFO - [2/5] Preprocessing...
2025-12-06 12:57:38,770 - src.preprocess - INFO - Loaded 33 slang/abbrev mappings
2025-12-06 12:57:38,772 - src.preprocess - INFO - Preprocessing 20 rows
2025-12-06 12:57:38,778 - src.preprocess - INFO - Preprocessing complete
2025-12-06 12:57:38,781 - src.utils - INFO - Checkpoint saved: /Users/vikranthnara/Documents/GitHub/Emotix/checkpoints/preprocessing_20251206_125738.parquet (20 rows)
2025-12-06 12:57:

Running complete Phase 2 pipeline...
This may take a few minutes to download the model on first run.



tokenizer_config.json:   0%|          | 0.00/294 [00:00<?, ?B/s]

config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

...

pytorch_model.bin:   0%|          | 0.00/329M [00:00<?, ?B/s]

Device set to use cpu
/Users/vikranthnara/Documents/GitHub/Emotix/.venv/lib/python3.12/site-packages/transformers/pipelines/text_classification.py:111: UserWarning: `return_all_scores` is now deprecated,  if want a similar functionality use `top_k=None` instead of `return_all_scores=True` or `top_k=1` instead of `return_all_scores=False`.
  warnings.warn(
2025-12-06 12:57:45,616 - src.modeling - INFO - Model loaded successfully
2025-12-06 12:57:45,616 - src.modeling - INFO - Running inference on 20 sequences
2025-12-06 12:57:45,616 - src.modeling - INFO - Predicting emotions for 20 texts


model.safetensors:   0%|          | 0.00/329M [00:00<?, ?B/s]

2025-12-06 12:57:46,033 - src.modeling - INFO - Completed predictions for 20 texts
2025-12-06 12:57:46,425 - src.modeling - INFO - Inference complete
2025-12-06 12:57:46,428 - src.utils - INFO - Checkpoint saved: /Users/vikranthnara/Documents/GitHub/Emotix/checkpoints/modeling_20251206_125746.parquet (20 rows)
2025-12-06 12:57:46,429 - src.pipeline - INFO - [5/5] Persistence...
2025-12-06 12:57:46,432 - src.persistence - INFO - Committed batch: 20 rows (total: 20)
2025-12-06 12:57:46,433 - src.persistence - INFO - Successfully wrote 20 rows to database
2025-12-06 12:57:46,433 - src.pipeline - INFO - Pipeline complete. Wrote 20 rows to database.



✓ Pipeline complete in 7.71 seconds
✓ Processed 20 records


## Step 6: View Results

Examine the emotion predictions and intensity scores.


In [3]:
# Display results
print("Sample predictions:\n")
display_cols = ['UserID', 'Text', 'NormalizedText', 'PrimaryEmotionLabel', 
                'IntensityScore_Primary', 'AmbiguityFlag']
print(df_results[display_cols].head(10))

# Summary statistics
print("\n" + "="*60)
print("Emotion Distribution:")
print("="*60)
print(df_results['PrimaryEmotionLabel'].value_counts())

print("\n" + "="*60)
print("Intensity Statistics:")
print("="*60)
print(df_results['IntensityScore_Primary'].describe())

print("\n" + "="*60)
print("Ambiguity Detection:")
print("="*60)
print(f"Ambiguous cases: {df_results['AmbiguityFlag'].sum()} / {len(df_results)}")
print(f"Percentage: {100 * df_results['AmbiguityFlag'].mean():.1f}%")


Sample predictions:

    UserID                                             Text  \
0  user001  Ugh, today was so stressful 😫 tbh I'm exhausted   
1  user001                         lol that's hilarious 😂😂😂   
2  user002           idk what to do anymore... feeling lost   
3  user001                 omg I can't believe it worked! 🎉   
4  user003                  wtf is wrong with me today? smh   
5  user002                    thx for being there for me ❤️   
6  user001                         brb need to take a break   
7  user003                         feeling anxious af rn...   
8  user002                imo this is getting better slowly   
9  user001                            yolo let's do this! 💪   

                                      NormalizedText PrimaryEmotionLabel  \
0  Ugh, today was so stressful tired_face to be h...             sadness   
1  laughing out loud that's hilarious face_with_t...             sadness   
2   i don't know what to do anymore. .. feeling lost     

## Step 7: Test Contextualization

Compare predictions with and without context.


In [4]:
# Test contextualization impact
persistence = MWBPersistence(db_path)
model = EmotionModel()

# Get a sample user's recent utterance
user_id = "user001"
history = persistence.fetch_history(user_id, limit=10)

if len(history) > 0:
    # Get most recent utterance
    latest = history.iloc[-1]
    utterance = latest['NormalizedText']
    
    print(f"Testing utterance: '{utterance}'\n")
    
    # Predict without context
    pred_no_context = model.predict_emotion(utterance)
    print("Without context:")
    print(f"  Label: {pred_no_context['label']}")
    print(f"  Intensity: {pred_no_context['intensity']:.3f}\n")
    
    # Predict with context
    from src.contextualize import create_sequence_for_model
    context_history = history.iloc[:-1]  # All but the last
    sequence = create_sequence_for_model(utterance, context_history)
    
    pred_with_context = model.predict_emotion(sequence)
    print("With context:")
    print(f"  Label: {pred_with_context['label']}")
    print(f"  Intensity: {pred_with_context['intensity']:.3f}\n")
    
    print(f"Context used: {len(context_history)} previous turns")
    if len(context_history) > 0:
        print(f"Sample context: {context_history['NormalizedText'].iloc[-1]}")
else:
    print("No history found for user")


2025-12-06 12:57:46,474 - src.persistence - INFO - Database schema initialized
2025-12-06 12:57:46,475 - src.modeling - INFO - Loading model: j-hartmann/emotion-english-distilroberta-base on cpu
Device set to use cpu
/Users/vikranthnara/Documents/GitHub/Emotix/.venv/lib/python3.12/site-packages/transformers/pipelines/text_classification.py:111: UserWarning: `return_all_scores` is now deprecated,  if want a similar functionality use `top_k=None` instead of `return_all_scores=True` or `top_k=1` instead of `return_all_scores=False`.
  warnings.warn(
2025-12-06 12:57:47,108 - src.modeling - INFO - Model loaded successfully
2025-12-06 12:57:47,111 - src.persistence - INFO - Fetched 10 records for user user001


Testing utterance: 'be right back need to take a break'

Without context:
  Label: sadness
  Intensity: 0.626

With context:
  Label: surprise
  Intensity: 0.953

Context used: 9 previous turns
Sample context: oh my god I can't believe it worked! party_popper


## Step 8: Synthetic Ambiguity Dataset

Generate and validate on synthetic ambiguous cases.


In [5]:
# Generate synthetic ambiguous cases
generator = AmbiguityGenerator()
ambiguous_df = generator.generate_ambiguous_cases(num_cases=20, include_context=True)

print(f"Generated {len(ambiguous_df)} ambiguous cases\n")
print("Sample ambiguous cases:")
print(ambiguous_df[['Text', 'PossibleLabels', 'Description']].head(5))

# Test model on ambiguous cases
print("\n" + "="*60)
print("Testing model on ambiguous cases:")
print("="*60)

model = EmotionModel()
ambiguous_results = []

for _, row in ambiguous_df.head(10).iterrows():
    text = row['Text']
    possible_labels = row['PossibleLabels']
    
    pred = model.predict_emotion(text)
    ambiguous_results.append({
        'Text': text,
        'PossibleLabels': possible_labels,
        'PredictedLabel': pred['label'],
        'Intensity': pred['intensity'],
        'IsCorrect': pred['label'].lower() in [l.lower() for l in possible_labels]
    })

results_df = pd.DataFrame(ambiguous_results)
print("\nResults:")
print(results_df[['Text', 'PossibleLabels', 'PredictedLabel', 'IsCorrect']])

accuracy = results_df['IsCorrect'].mean()
print(f"\nAccuracy on ambiguous cases: {accuracy:.2%}")


2025-12-06 12:57:47,198 - src.validation - INFO - Generating 20 ambiguous cases
2025-12-06 12:57:47,199 - src.validation - INFO - Generated 20 ambiguous cases
2025-12-06 12:57:47,201 - src.modeling - INFO - Loading model: j-hartmann/emotion-english-distilroberta-base on cpu


Generated 20 ambiguous cases

Sample ambiguous cases:
           Text                 PossibleLabels  \
0      I'm fine      [neutral, sadness, anger]   
1      Whatever      [neutral, anger, sadness]   
2  That's great          [joy, sarcasm, anger]   
3  I don't know  [confusion, sadness, anxiety]   
4         Maybe  [neutral, anxiety, confusion]   

                                Description  
0    Sarcastic 'fine' can indicate distress  
1     Dismissive response can hide emotions  
2        Sarcastic positive can be negative  
3  Uncertainty can indicate multiple states  
4            Hedging can mask true feelings  

Testing model on ambiguous cases:


Device set to use cpu
/Users/vikranthnara/Documents/GitHub/Emotix/.venv/lib/python3.12/site-packages/transformers/pipelines/text_classification.py:111: UserWarning: `return_all_scores` is now deprecated,  if want a similar functionality use `top_k=None` instead of `return_all_scores=True` or `top_k=1` instead of `return_all_scores=False`.
  warnings.warn(
2025-12-06 12:57:47,863 - src.modeling - INFO - Model loaded successfully



Results:
           Text                   PossibleLabels PredictedLabel  IsCorrect
0      I'm fine        [neutral, sadness, anger]        neutral       True
1      Whatever        [neutral, anger, sadness]        neutral       True
2  That's great            [joy, sarcasm, anger]            joy       True
3  I don't know    [confusion, sadness, anxiety]       surprise      False
4         Maybe    [neutral, anxiety, confusion]        neutral       True
5       I guess  [neutral, uncertainty, anxiety]        neutral       True
6      I'm okay      [neutral, sadness, anxiety]        neutral       True
7  It's nothing        [neutral, sadness, anger]        neutral       True
8      I'm good          [joy, neutral, sarcasm]            joy       True
9     I'm tired       [neutral, sadness, stress]        sadness       True

Accuracy on ambiguous cases: 90.00%


## Step 9: Post-Processing Analysis

Analyze the impact of post-processing rules on predictions.


In [ ]:
# Test post-processing on results
from src.postprocess import apply_post_processing

if 'PrimaryEmotionLabel' in df_results.columns:
    df_postprocessed = apply_post_processing(df_results)
    
    print("Post-Processing Results:")
    print("="*60)
    print(f"Total predictions: {len(df_postprocessed)}")
    print(f"Overrides applied: {df_postprocessed['WasOverridden'].sum()}")
    print(f"Flagged for review: {df_postprocessed['NeedsReview'].sum()}")
    
    if df_postprocessed['WasOverridden'].sum() > 0:
        print("\nOverrides Applied:")
        overrides = df_postprocessed[df_postprocessed['WasOverridden'] == 1]
        for idx, row in overrides.head(5).iterrows():
            print(f"\n  '{row['NormalizedText'][:50]}...'")
            print(f"    {row['PrimaryEmotionLabel']} ({row['IntensityScore_Primary']:.2f}) → {row['CorrectedLabel']} ({row['CorrectedIntensity']:.2f})")
    
    print("\n" + "="*60)
    print("Emotion Distribution (After Post-Processing):")
    print("="*60)
    if 'CorrectedLabel' in df_postprocessed.columns:
        print(df_postprocessed['CorrectedLabel'].value_counts())
    else:
        print(df_postprocessed['PrimaryEmotionLabel'].value_counts())
else:
    print("No predictions found. Run the pipeline first.")


## Summary

✅ **Complete Pipeline**: All 5 layers integrated  
✅ **Emotion Classification**: DistilRoBERTa-based model for emotion labels  
✅ **Intensity Prediction**: Softmax probabilities → intensity scores (0.0-1.0)  
✅ **Contextualization**: Multi-turn sequences for ambiguity resolution  
✅ **Ambiguity Detection**: Automatic flagging of ambiguous cases  
✅ **Validation**: Synthetic dataset generator for testing  

**Next Steps:**
- Fine-tune model on labeled MWB dataset
- Expand ambiguity detection with more sophisticated methods
- Implement human-in-the-loop feedback mechanism
- Add real-time inference API
